## Hands-on session 1.2 - advection model continued: refining with built-in models - Solutions

This notebook continues from session 1.1, moving away from building all terms by hand and utilizing the common_models module to refactor the existing code. The concept of adding diagnostic manipulators for individual variables to the run is also covered here.

Demonstrated concepts:

- Common models for advection and pressure gradients 
- Setting outflow boundary condition on advection models
- Querying the wrapper on added models and terms
- Diagnostic variables and manipulators
- Using the built-in dashboard to visualise fluid variable evolution

In [ ]:
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.common_models as cm
import RMK_support.IO_support as io

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

### Pre-model initialization

This follows as in session 1.1

In [ ]:
#Wrapper initialization
rk = RKWrapper()

#I/O setuo
rk.jsonFilepath = "./config.json" 
hdf5Filepath = "./RMKOutput/day_1_1/"
rk.setHDF5Path(hdf5Filepath) 

#MPI setup
rk.setMPIData(numProcsX=4)

# Normalization
rk.setNormDensity(1.0e19) 
rk.setNormTemperature(10.0) 
rk.setNormRefZ(1.0) 

#Grid initialization
xGridWidths = 0.025*np.ones(512)
gridObj = Grid(xGridWidths, interpretXGridAsWidths=True)
rk.grid = gridObj

#Variables

n = 1 + np.exp(-(gridObj.xGrid-np.mean(gridObj.xGrid))**2) 
T = np.ones(len(gridObj.xGrid)) 

rk.addVarAndDual('n',n,isCommunicated=True) 
rk.addVarAndDual('T',T,isDerived=True) 
rk.addVarAndDual('G',isCommunicated=True,primaryOnDualGrid=True) 
rk.addVarAndDual('u',isDerived=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["G_dual","n_dual"]),primaryOnDualGrid=True,isCommunicated=True)
rk.addVar('time',isDerived=True,isScalar=True)

### Models and Terms

The module `common_models` is supplied to speed up the initialization of models and terms that show up often. 

**NOTE**: Many models in `common_models` assume the default normalization as available from `sk_normalization`.

The advection model for the density variable can be added using the `common_models` function `staggeredAdvection`:

In [ ]:
cm.staggeredAdvection?

To add outflow we set the `rightOutflow` argument to true, and since the `advectionSpeed` argument is required we supply the variable 'u'. Since the function calls for the variable on the regular grid, the `_dual` suffix is dropped. 

**NOTE**: The default outflow boundary condition linearly extrapolates the `advectionSpeed` variable to the domain boundaries and uses that as the outflow velocity (making sure that its projection on the domain normal is positive in order to avoid spurious inflow)

In [ ]:
rk.addModel(cm.staggeredAdvection("nAdvection","n","G_dual",rightOutflow=True,advectionSpeed="u"))

Similarly, the pressure gradient can be added in a one-liner using the `staggeredPressureGrad` function


In [ ]:
cm.staggeredPressureGrad?

Here we need to supply a temperature variable (hence its inclusion in session 1.1!) and species mass (this time in SI!)

In [ ]:
massRatio = 1/1836

rk.addModel(cm.staggeredPressureGrad('pGrad',"G_dual","n","T",9.1093837e-31/massRatio))

The momentum equation with momentum advection is given as 

$m_i \frac{\partial \Gamma}{\partial t} = - m_i\frac{\partial (\Gamma u)}{\partial x} - \frac{\partial (nkT)}{\partial x}$

and the momentum advection term is easily normalized by remembering that $u_0=x_0/t_0$. 

In order to implement it, we must remember that we're advecting a variable on the dual grid. Looking at the documentation of `staggeredAdvection` we see that we need to specify this directly, and give the staggered advection speed in order to reconstruct the flux. Since the flux is reconstructed, the `fluxVar` argument is unused


In [ ]:
rk.addModel(cm.staggeredAdvection("GAdvection",advectedVar="G_dual",fluxVar="",rightOutflow=True,advectionSpeed="u",staggeredAdvectionSpeed="u_dual",staggeredAdvectedVar=True))

### Time integration options

These are set in the same way as in session 1.1


In [ ]:
rk.addIntegrator("BE",sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=['n','G_dual']) )

rk.setIntegratorGlobalData(initialTimestep=0.1) 

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(10000)
rk.setFixedStepOutput(200)

### Query functions on the wrapper

The following functions are useful when inspecting already constructed wrappers:

In [ ]:
rk.varList()

In [ ]:
rk.modelTags()

In [ ]:
rk.getTermsThatEvolveVar('n')

For more information on the wrapper see the [rk_wrapper](https://remkit1d-python.readthedocs.io/en/latest/RMK_support.html#module-RMK_support.rk_wrapper) documentation.

### Adding diagnostic variables

In many cases we would like to inspect individual terms in various equations. One way of doing this in ReMKiT1D for custom models is to attach a set of manipulators to a variable name. This is all abstracted in the wrapper function `addTermDiagnosisForVars`

In [ ]:
rk.addTermDiagnosisForVars(['n','G_dual'])

This has now added new variables to evaluate terms that evolve the density adn flux and we can check this

In [ ]:
rk.varList()

### Create config 

In [ ]:
rk.writeConfigFile()

### Set global plotting options

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

### Load data from ReMKiT1D output files

In [ ]:
numFiles = 50
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

### Explore data with built-in dashboard

`RMK_support` offers a simple built-in dashboard to quickly explore variables 

In [ ]:
import panel as pn 
import RMK_support.dashboard_support as ds

pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show() # Removing .show() should display the dashboard inline - this can be buggy in some situations


Launching server at http://localhost:36653
